In [ ]:
# download nltk stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
 !apt-get install git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.13).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [ ]:
!pip git clone https://github.com/facebookresearch/fastText.git

ERROR: unknown command "git"


In [ ]:
# Install a particular version of `google-cloud-storage` because (oddly enough) 
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 KB 3.4 MB/s eta 0:00:00


In [ ]:
!python download_model.py en

python3: can't open file 'download_model.py': [Errno 2] No such file or directory


In [ ]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()

In [ ]:
# install ngrok to emulate public IP / address
!wget -N https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip

for details.

--2023-01-12 08:17:23--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 54.161.241.46, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  4.77MB/s    in 2.8s    

2023-01-12 08:17:27 (4.77 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
# TODO: sign up for an ngrok account
# then put your ngrok token below, uncomment, and execute
!./ngrok authtoken 22KeqcMQYpd2iRnsaZZOzlyaNw0_5NAk69pXRUMvfrf1oCNpU

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# install a ngrok python package and a version of flask that works with it in 
# colab
!pip -q install flask-ngrok
!pip -q install flask==0.12.2
!pip -q install flask_restful
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 5.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Run the app

In [ ]:
# you need to upload your implementation of search_app.py
!pip install fasttext==0.9.2
import search_frontend as se

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

# uncomment the code below and execute to reload the module when you make 
# changes to search_frontend.py (after you upload again).
import importlib
importlib.reload(se)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'search_frontend' from '/content/search_frontend.py'>

In [ ]:
from flask_ngrok import run_with_ngrok
run_with_ngrok(se.app) 
se.app.run()

FileNotFoundError: ignored

# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [ ]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [ ]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [ ]:
import numpy as np
import random
import requests
from time import time
from scipy.optimize import minimize

url = 'http://34.27.134.71:8080'

# Define a function to calculate the total AP for the current parameter values
def calculate_ap(params):
    ap_total = 0
    for q, true_wids in queries.items():
        try:
            res = requests.get(url + '/search', {'query': q,'b':params[0],'k1':params[1],'k3':params[2],'k4':params[3],'k5':params[4]}, timeout=35)
            if res.status_code == 200:
                pred_wids, _ = zip(*res.json())
                ap = average_precision(true_wids, pred_wids)
                ap_total += ap
        except:
            pass
    print(ap_total)
    return -ap_total # negate because we want to maximize AP

initial_guess_1 = [90, 1350, 8, 4.5, 30]
initial_guess_2 = [80, 1250, 7, 4.0, 25]
initial_guess_3 = [100, 1450, 9, 5.0, 35]

res_1 = minimize(calculate_ap, initial_guess_1, method='BFGS')
res_2 = minimize(calculate_ap, initial_guess_2, method='BFGS')
res_3 = minimize(calculate_ap, initial_guess_3, method='BFGS')

# Compare the results
print(f'Initial guess 1: {res_1.x} - {res_1.fun}')
print(f'Initial guess 2: {res_2.x} - {res_2.fun}')
print(f'Initial guess 3: {res_3.x} - {res_3.fun}')



In [ ]:
!pip install optuna


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import random
import requests
from time import time
from scipy.optimize import differential_evolution
from multiprocessing import Pool
import optuna

url = 'http://34.27.134.71:8080'


def objective(trial):
    b = trial.suggest_uniform('b', 50, 150)
    k1 = trial.suggest_uniform('k1', 1000, 1500)
    k3 = trial.suggest_uniform('k3', 5, 10)
    k4 = trial.suggest_uniform('k4', 3, 5)
    k5 = trial.suggest_uniform('k5', 20, 40)
    pv=trial.suggest_uniform('pv', 1,2)
    pr=trial.suggest_uniform('pr', 0.1,0.7)

    params = [b, k1, k3, k4, k5,pv,pr]
    ap_total = 0
    for q, true_wids in queries.items():
        try:
            res = requests.get(url + '/search', {'query': q,'b':params[0],'k1':params[1],'k3':params[2],'k4':params[3],'k5':params[4],'pr':params[5],'pv':params[6]}, timeout=35)
            if res.status_code == 200:
                pred_wids, _ = zip(*res.json())
                ap = average_precision(true_wids, pred_wids)
                ap_total += ap
        except:
            pass
    print(ap_total)
    print(b,k1,k3,k4,k5)
    return -ap_total

study = optuna.create_study()
study.optimize(objective, n_trials=100)


[I 2023-01-15 15:48:55,812] A new study created in memory with name: no-name-c70e91d4-164f-49e9-ae0d-f9b95766408f
<ipython-input-4-0d77c8b8b547>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  b = trial.suggest_uniform('b', 50, 150)
<ipython-input-4-0d77c8b8b547>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  k1 = trial.suggest_uniform('k1', 1000, 1500)
<ipython-input-4-0d77c8b8b547>:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  k3 = trial.suggest_uniform('k3', 5, 10)
<ipyt

18.427999999999997
119.96044879029097 1322.1374621795574 5.176876506799044 3.5410459581419618 22.532376366838825


[I 2023-01-15 15:52:03,469] Trial 1 finished with value: -17.548999999999996 and parameters: {'b': 98.08172676637072, 'k1': 1133.9088951813621, 'k3': 9.433776529441095, 'k4': 4.98083553572758, 'k5': 25.521676314613195, 'pv': 1.353791688360574, 'pr': 0.2857886134018864}. Best is trial 0 with value: -18.427999999999997.


17.548999999999996
98.08172676637072 1133.9088951813621 9.433776529441095 4.98083553572758 25.521676314613195


[I 2023-01-15 15:53:36,956] Trial 2 finished with value: -17.436 and parameters: {'b': 128.37971809862313, 'k1': 1270.203643754613, 'k3': 8.792417786060248, 'k4': 4.325525516165804, 'k5': 37.07064279532851, 'pv': 1.6189025947345446, 'pr': 0.24593883240472098}. Best is trial 0 with value: -18.427999999999997.


...

0
146.31947094568557 1180.5801629118782 6.522715400746829 4.782400436352763 28.768573545315697
0
140.40821110737437 1235.8057395433361 5.402748343472751 4.903442075737452 30.182476390353862


[I 2023-01-15 17:35:01,657] Trial 97 finished with value: 0.0 and parameters: {'b': 129.8067064544983, 'k1': 1446.785585282481, 'k3': 5.160837927498797, 'k4': 3.9792958889300105, 'k5': 25.162935660727875, 'pv': 1.264569107667851, 'pr': 0.33223694253430625}. Best is trial 65 with value: -19.152.
[I 2023-01-15 17:35:01,768] Trial 98 finished with value: 0.0 and parameters: {'b': 148.2397705935053, 'k1': 1054.865878659327, 'k3': 5.9494402564538795, 'k4': 3.9184531371306197, 'k5': 32.605778067745064, 'pv': 1.2019303966619472, 'pr': 0.6858361344986453}. Best is trial 65 with value: -19.152.


0
129.8067064544983 1446.785585282481 5.160837927498797 3.9792958889300105 25.162935660727875
0
148.2397705935053 1054.865878659327 5.9494402564538795 3.9184531371306197 32.605778067745064


[I 2023-01-15 17:35:01,883] Trial 99 finished with value: 0.0 and parameters: {'b': 144.9075995218242, 'k1': 1112.3184346386663, 'k3': 5.543003615200531, 'k4': 4.941284199292853, 'k5': 25.85800604506589, 'pv': 1.3688932044561632, 'pr': 0.1053857328604435}. Best is trial 65 with value: -19.152.


0
144.9075995218242 1112.3184346386663 5.543003615200531 4.941284199292853 25.85800604506589


In [ ]:
import optuna
b_range = [50,150]
k1_range = [500,2000]
k3_range = [5,30]
k5_range = [5,60]
k4_range = [1,16]
pr_range = [1.0,2.0]
pv_range = [0.1,0.7]
def objective(trial):
    b = trial.suggest_uniform('b', 50, 150)
    k1 = trial.suggest_uniform('k1', 1000, 1500)
    k3 = trial.suggest_uniform('k3', 5, 10)
    k4 = trial.suggest_uniform('k4', 3, 5)
    k5 = trial.suggest_uniform('k5', 20, 40)
    pv=trial.suggest_uniform('pv', 1,2)
    pr=trial.suggest_uniform('pr', 0.1,0.7)

    params = [b, k1, k3, k4, k5,pv,pr]
    ap_total = 0
    for q, true_wids in queries.items():
        try:
            res = requests.get(url + '/search', {'query': q,'b':params[0],'k1':params[1],'k3':params[2],'k4':params[3],'k5':params[4],'pr':params[5],'pv':params[6]}, timeout=35)
            if res.status_code == 200:
                pred_wids, _ = zip(*res.json())
                ap = average_precision(true_wids, pred_wids)
                ap_total += ap
        except:
            pass
    return -ap_total

study = optuna.create_study
study.optimize(objective, n_trials=100)

AttributeError: ignored

In [ ]:
print("Best parameters:", params)
print(total_ap - prev_ap)

NameError: ignored

In [ ]:
import random
import requests
from time import time
url = 'http://104.197.128.88:8080'
b_range = [50,150]
k1_range = [500,2000]
k3_range = [5,30]
k5_range = [5,60]
k4_range = [1,16]
best_ap = 0
best_params = {}
for i in range(100):
  print(best_ap)
  print(i)
  b = random.choice(b_range)
  k1 = random.choice(k1_range)
  k4= random.choice(k4_range)
  k3= random.choice(k3_range)
  k5= random.choice(k5_range)
  ap_total =0
  qs_res = []
  for q, true_wids in queries.items():
    duration, ap = None, None
    t_start = time()
    try:
      res = requests.get(url + '/search', {'query': q,'b':b,'k1':k1,'k3':k3,'k4':k4,'k5':k5}, timeout=35)
      duration = time() - t_start
      if res.status_code == 200:
        pred_wids, _ = zip(*res.json())
        ap = average_precision(true_wids, pred_wids)
        ap_total += ap
    except:
      pass
    qs_res.append((q, duration, ap))
  if best_ap < ap_total:
    best_ap = ap_total
    best_params[best_ap] = (k1,b,k3,k4,k5)
    
    
  


0
0
12.449
1
12.449
2
14.08
3
14.08
4
16.049
5
16.049
6
16.049
7
16.049
8
16.049
9
16.762
10
16.762
11
16.762
12
16.944999999999997
13
16.944999999999997
14
16.944999999999997
15
16.944999999999997
16
16.944999999999997
17
16.944999999999997
18
16.944999999999997
19
16.944999999999997
20
16.944999999999997
21
16.944999999999997
22
16.944999999999997
23
16.944999999999997
24
16.944999999999997
25
16.944999999999997
26
16.944999999999997
27
16.944999999999997
28
16.944999999999997
29
16.944999999999997
30
16.944999999999997
31
16.944999999999997
32
16.944999999999997
33
16.944999999999997
34
16.944999999999997
35
16.944999999999997
36
16.944999999999997
37
16.944999999999997
38
16.944999999999997
39
16.944999999999997
40
16.944999999999997
41
16.944999999999997
42
16.944999999999997
43
16.944999999999997
44
16.944999999999997
45
16.944999999999997
46
16.944999999999997
47
16.944999999999997
48
16.944999999999997
49
16.944999999999997
50
16.944999999999997
51
16.944999999999997
52
16.9449

In [ ]:
print( qs_res)
print(best_ap/len(queries))
print(best_params)


[('best marvel movie', None, None), ('How do kids come to world?', None, None), ('Information retrieval', None, None), ('LinkedIn', None, None), ('How to make coffee?', None, None), ('Ritalin', None, None), ('How to make wine at home?', None, None), ('Most expensive city in the world', None, None), ('India', None, None), ('how to make money fast?', None, None), ('Netflix', None, None), ('Apple computer', None, None), ('The Simpsons', None, None), ('World cup', None, None), ('How to lose weight?', None, None), ('Java', None, None), ('Air Jordan', None, None), ('how to deal with depression?', None, None), ('How do you make gold', None, None), ('Marijuana', None, None), ('How to make hummus', None, None), ('Winter', None, None), ('Rick and Morty', None, None), ('Natural Language processing', None, None), ('World Cup 2022', None, None), ('Dolly the sheep', None, None), ('Cigarettes', None, None), ('What is the best place to live in?', None, None), ('Elon musk', None, None), ('How do you br